In [14]:
import torch
# GPUの名称、CUDA Compute Capabilityを取得
print(torch.cuda.get_device_name())

NVIDIA GeForce RTX 2080 Ti


In [ ]:
import cupy as cp
import numpy as np
import pandas as pd
import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import scipy.stats as st
import pickle
import optuna
import re
import gc

from lightgbm import LGBMClassifier

from sklearn import metrics
from sklearn.tree import plot_tree
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.impute import SimpleImputer
from sklearn_pandas import DataFrameMapper
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import log_loss
from sklearn.metrics import mean_squared_log_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.experimental import enable_iterative_imputer
from sklearn.metrics import roc_auc_score
from sklearn.datasets import make_moons
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import mutual_info_classif
from sklearn.experimental import enable_iterative_imputer

from keras.layers import Dense, Dropout
from keras.models import Sequential

from six import StringIO

from IPython.display import Image
import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.api as sm
import math
from scipy import stats
import numpy as np
import pandas as pd
import tensorflow as tf

from datetime import datetime
from tensorflow import feature_column
from tensorflow.keras import layers

from tqdm import tqdm

# sklearn preprocessing for dealing with categorical variables
from sklearn.preprocessing import LabelEncoder

# File system manangement
import os

# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')

import xgboost as xgb
from sklearn.metrics import log_loss
from xgboost.sklearn import XGBClassifier

from Modules import Modules
from Baseline import Baseline
from FeatureEngineering2 import FeatureEngineering2


%matplotlib inline

print("OK\n")

## 共通関数定義

In [72]:
"""
params_base = {
            'boosting_type': 'gbdt'
            ,'objective': 'binary'
            ,'metric': 'auc'
            ,'verbosity': -1
            ,'learning_rate': 0.05
            ,'n_estimators': 100000
            ,'random_state':123
            ,'bagging_freq': 1   
}
"""
params_base = {
    'boosting_type': 'gbdt'
    ,'objective': 'binary'
    ,'metric': 'auc'
    ,'nthread': -1
    ,'n_estimators': 5000
    ,'learning_rate': 0.01
    ,'max_depth': 11
    ,'colsample_bytree': 0.613
    ,'subsample': 0.708
    ,'max_bin': 407
    ,'reg_alpha': 3.564
    ,'reg_lambda': 4.930
    ,'min_child_samples': 165
    ,'silent': -1
    ,'verbose': -1
    ,'importance_type': 'gain'
}



def objective(trial):
    # 探索するパラメータ
    param_tuning = {
            "num_leaves": trial.suggest_int("num_leaves", 8, 256)
            ,"min_child_samples": trial.suggest_int("min_child_samples", 5, 200)
            ,"min_sum_hessian_in_leaf": trial.suggest_float("min_sum_hessian_in_leaf", 1e-5, 1e-2, log=True)
            ,"feature_fraction": trial.suggest_float("feature_fraction", 0.5, 1.0)
            ,"bagging_fraction": trial.suggest_float("bagging_fraction", 0.5, 1.0)
            ,"lambda_l1": trial.suggest_float("lambda_l1", 1e-2, 1e+2, log=True)
            ,"lambda_l2": trial.suggest_float("lambda_l2", 1e-2, 1e+2, log=True)
    }
    param_tuning.update(params_base)

    # モデル学習・評価:
    list_metrics = []
    cv = list(StratifiedKFold(n_splits=5, shuffle=True, random_state=123).split(x_train, y_train))
    list_fold = [0]
    for nfold in list_fold:
        idx_tr, idx_va = cv[nfold][0], cv[nfold][1]
        x_tr, y_tr = x_train.loc[idx_tr, :], y_train[idx_tr]
        x_va, y_va = x_train.loc[idx_va, :], y_train[idx_va]
        model = lgb.LGBMClassifier(**params)
        model.fit(x_tr
                    ,y_tr
                    ,eval_set=[(x_tr, y_tr), (x_va, y_va)]
                    ,early_stopping_rounds=100
                    ,verbose=0
        )
        y_va_pred = model.predict_proba(x_va)[:,1]
        metric_va = roc_auc_score(y_va, y_va_pred)
        list_metrics.append(metric_va)

    # 評価指数の算出
    metrics = np.mean(list_metrics)

    return metrics

## インスタンス生成

In [2]:
baseline = Baseline('SK_ID_CURR')
modules = Modules('SK_ID_CURR')
FeatureEngineering2 = FeatureEngineering2('SK_ID_CURR')

## 特徴量生成

In [3]:
### application
df = FeatureEngineering2.application()
df.head()

"Application_Train_Test" final shape: (356250, 309)


,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,APP_EXT_SOURCE_2*DAYS_EMPLOYED,APP_EXT_SOURCE_3*DAYS_EMPLOYED,APPS_GOODS_INCOME_RATIO,APPS_CNT_FAM_INCOME_RATIO,APPS_INCOME_EMPLOYED_RATIO,CREDIT_TO_GOODS_RATIO_2,APP_AMT_INCOME_TOTAL_12_AMT_ANNUITY_ratio,APP_INCOME_TO_EMPLOYED_RATIO,APP_DAYS_LAST_PHONE_CHANGE_DAYS_EMPLOYED_ratio,APP_DAYS_EMPLOYED_DAYS_BIRTH_diff
0,0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,...,-167.498254,-88.782372,1.733333,202500.0,-317.896389,1.158397,-7825.5,-317.896389,1.780220,8824.0
1,1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,...,-739.227981,NaN,4.183333,135000.0,-227.272727,1.145199,-13198.5,-227.272727,0.696970,15577.0
2,2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,...,-125.080219,-164.152505,2.000000,67500.0,-300.000000,1.000000,-1125.0,-300.000000,3.622222,18821.0
3,3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,...,-1976.692297,NaN,2.200000,67500.0,-44.422507,1.052803,-18436.5,-44.422507,0.203027,15966.0
4,4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,...,-980.478916,NaN,4.222222,121500.0,-39.993417,1.000000,-11740.5,-39.993417,0.364055,16894.0


In [4]:
### bureau_balance
df = df.merge(FeatureEngineering2.bureau_bb(), how='left', on='SK_ID_CURR')
df.head()

"Bureau/Bureau Balance" final shape: (305811, 200)


,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CLOSED_BUREAU_CREDIT_DEBT_RATIO_MIN,CLOSED_BUREAU_CREDIT_DEBT_RATIO_MAX,CLOSED_BUREAU_CREDIT_DEBT_RATIO_MEAN,CLOSED_DEBT_CREDIT_DIFF_MIN,CLOSED_DEBT_CREDIT_DIFF_MAX,CLOSED_DEBT_CREDIT_DIFF_MEAN,CLOSED_BUREAU_IS_DPD_MEAN,CLOSED_BUREAU_IS_DPD_SUM,CLOSED_BUREAU_IS_DPD_OVER120_MEAN,CLOSED_BUREAU_IS_DPD_OVER120_SUM
0,0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,...,0.0,0.0,0.0,0.0,135000.0,85245.0,0.0,0.0,0.0,0.0
1,1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,...,0.0,0.0,0.0,22248.0,112500.0,69133.5,0.0,0.0,0.0,0.0
2,2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,...,0.0,0.0,0.0,94500.0,94537.8,94518.9,0.0,0.0,0.0,0.0
3,3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,...,0.0,0.0,0.0,146250.0,146250.0,146250.0,0.0,0.0,0.0,0.0


In [5]:
### previos_application
df = df.merge(FeatureEngineering2.previous_application(), how='left', on='SK_ID_CURR')
df.head()

"Previous Applications" final shape: (338857, 321)


,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,REFUSED_PREV_GOODS_DIFF_MEAN,REFUSED_PREV_GOODS_DIFF_MAX,REFUSED_PREV_GOODS_DIFF_SUM,REFUSED_PREV_GOODS_APPL_RATIO_MEAN,REFUSED_PREV_GOODS_APPL_RATIO_MAX,REFUSED_DAYS_LAST_DUE_DIFF_MEAN,REFUSED_DAYS_LAST_DUE_DIFF_MAX,REFUSED_DAYS_LAST_DUE_DIFF_SUM,REFUSED_SIMPLE_INTERESTS_MEAN,REFUSED_SIMPLE_INTERESTS_MAX
0,0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,...,0.0,0.0,0.0,1.0,1.0,NaN,NaN,0.0,0.015231,0.015231
4,4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
### pos_cash
df = df.merge(FeatureEngineering2.pos_cash(), how='left', on='SK_ID_CURR')
df.head()

"Pos-Cash" balance final shape: (337252, 46)


,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,POS_NAME_CONTRACT_STATUS_Returned to the store_MEAN,POS_NAME_CONTRACT_STATUS_Signed_MEAN,POS_NAME_CONTRACT_STATUS_XNA_MEAN,POS_NAME_CONTRACT_STATUS_nan_MEAN,POS_COUNT,POS_LOAN_COMPLETED_MEAN,POS_COMPLETED_BEFORE_MEAN,POS_REMAINING_INSTALMENTS,POS_REMAINING_INSTALMENTS_RATIO,LATE_PAYMENT_SUM
0,0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,...,0.000000,0.000000,0.0,0.0,19.0,0.000000,0.0,6.0,0.250000,0.0
1,1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,...,0.000000,0.000000,0.0,0.0,28.0,0.250000,1.0,1.0,0.083333,0.0
2,2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,...,0.000000,0.000000,0.0,0.0,4.0,0.250000,1.0,0.0,0.000000,NaN
3,3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,...,0.047619,0.000000,0.0,0.0,21.0,0.458333,2.0,3.0,0.250000,0.0
4,4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,...,0.000000,0.015152,0.0,0.0,66.0,0.209402,1.0,14.0,0.641667,0.0


In [7]:
### instllment_payment
df = df.merge(FeatureEngineering2.installment(), how='left', on='SK_ID_CURR')
df.head()

"Installments Payments" final shape: (339587, 85)


,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,INS_D365PAYMENT_PERC_MAX,INS_D365DPD_DIFF_MEAN,INS_D365DPD_DIFF_MIN,INS_D365DPD_DIFF_MAX,INS_D365DPD_MEAN,INS_D365DPD_SUM,INS_D365INS_IS_DPD_UNDER_120_MEAN,INS_D365INS_IS_DPD_UNDER_120_SUM,INS_D365INS_IS_DPD_OVER_120_MEAN,INS_D365INS_IS_DPD_OVER_120_SUM
0,0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,...,1.0,-17.363636,-24.0,-12.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,...,1.0,-6.181818,-23.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,...,1.0,-2.250000,-4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
### credit_card_balance
df = df.merge(FeatureEngineering2.credit_card(), how='left', on='SK_ID_CURR')
df.head()

"Credit Card Balance" final shape: (103558, 284)


,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,INS_48M_SK_DPD_MAX,INS_48M_SK_DPD_SUM,INS_48M_DRAWING_LIMIT_RATIO_MIN,INS_48M_DRAWING_LIMIT_RATIO_MAX,INS_48M_LATE_PAYMENT_MEAN,INS_48M_LATE_PAYMENT_SUM,INS_48M_CARD_IS_DPD_UNDER_120_MEAN,INS_48M_CARD_IS_DPD_UNDER_120_SUM,INS_48M_CARD_IS_DPD_OVER_120_MEAN,INS_48M_CARD_IS_DPD_OVER_120_SUM
0,0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,...,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0
4,4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
### 
df = FeatureEngineering2.data_post_processing(df)
print('='*50, '\n')
print('---=> df final shape:', df.shape, ' <=---', '\n')
print('=' * 50)
df.head()

---=> the DATA POST-PROCESSING is beginning, the dataset has 1243 features
names of feature are renamed
---=> pandas data types of features in the dataset are converted for a reduced memory usage
---=> 1199 features are remained after removing non-informative features
164 features are eliminated by LightGBM classifier with an 0.7 auc score limit in step I
---=> 1035 features are remained after removing features not interesting for LightGBM classifier
---=> 46 features are generated with the risk_groupanizer
---=> the DATA POST-PROCESSING is ended!, now the dataset has a total 1081 features

---=> df final shape: (356250, 1081)  <=--- 



,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CLOSED_CNT_CREDIT_PROLONG_SUM_high_risk,CLOSED_CNT_CREDIT_PROLONG_SUM_low_risk,PREV_PREV_GOODS_DIFF_MAX_high_risk,PREV_PREV_GOODS_DIFF_MAX_low_risk,POS_COMPLETED_BEFORE_MEAN_high_risk,POS_COMPLETED_BEFORE_MEAN_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk
0,0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,...,0,1,1,0,1,0,0,0,0,0
1,1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,...,0,1,1,0,0,1,0,0,0,0
2,2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,...,0,1,1,0,0,1,0,0,0,0
3,3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,...,0,0,1,0,0,1,1,0,0,0
4,4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,...,0,1,1,0,0,1,0,0,0,0


## 学習

### 001 baseline

In [25]:
# split train, and test datasets
app_train = df[df['TARGET'].notnull()]
app_test = df[df['TARGET'].isnull()]

print(app_test.shape)
app_test.head()

(48744, 1081)


,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CLOSED_CNT_CREDIT_PROLONG_SUM_high_risk,CLOSED_CNT_CREDIT_PROLONG_SUM_low_risk,PREV_PREV_GOODS_DIFF_MAX_high_risk,PREV_PREV_GOODS_DIFF_MAX_low_risk,POS_COMPLETED_BEFORE_MEAN_high_risk,POS_COMPLETED_BEFORE_MEAN_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk
307506,0,100001,NaN,1,0,0,0,135000.0,568800.0,20560.5,...,0,1,1,0,1,0,0,0,0,0
307507,1,100005,NaN,0,0,0,0,99000.0,222768.0,17370.0,...,0,1,1,0,0,1,0,0,0,0
307508,2,100013,NaN,0,1,0,0,202500.0,663264.0,69777.0,...,0,1,1,0,0,1,1,0,1,0
307509,3,100028,NaN,1,0,0,2,315000.0,1575000.0,49018.5,...,0,1,1,0,0,1,1,0,1,0
307510,4,100038,NaN,0,1,1,1,180000.0,625500.0,32067.0,...,0,0,1,0,1,0,0,0,0,0


In [26]:
### indexを削除
app_train=app_train.reset_index()
app_test=app_test.reset_index()
app_test.head()

,level_0,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,...,CLOSED_CNT_CREDIT_PROLONG_SUM_high_risk,CLOSED_CNT_CREDIT_PROLONG_SUM_low_risk,PREV_PREV_GOODS_DIFF_MAX_high_risk,PREV_PREV_GOODS_DIFF_MAX_low_risk,POS_COMPLETED_BEFORE_MEAN_high_risk,POS_COMPLETED_BEFORE_MEAN_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk
0,307506,0,100001,NaN,1,0,0,0,135000.0,568800.0,...,0,1,1,0,1,0,0,0,0,0
1,307507,1,100005,NaN,0,0,0,0,99000.0,222768.0,...,0,1,1,0,0,1,0,0,0,0
2,307508,2,100013,NaN,0,1,0,0,202500.0,663264.0,...,0,1,1,0,0,1,1,0,1,0
3,307509,3,100028,NaN,1,0,0,2,315000.0,1575000.0,...,0,1,1,0,0,1,1,0,1,0
4,307510,4,100038,NaN,0,1,1,1,180000.0,625500.0,...,0,0,1,0,1,0,0,0,0,0


In [27]:
### indexを削除
app_train = app_train.drop(columns=['index'])
app_test = app_test.drop(columns=['index'])
app_test.head()

,level_0,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CLOSED_CNT_CREDIT_PROLONG_SUM_high_risk,CLOSED_CNT_CREDIT_PROLONG_SUM_low_risk,PREV_PREV_GOODS_DIFF_MAX_high_risk,PREV_PREV_GOODS_DIFF_MAX_low_risk,POS_COMPLETED_BEFORE_MEAN_high_risk,POS_COMPLETED_BEFORE_MEAN_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk
0,307506,100001,NaN,1,0,0,0,135000.0,568800.0,20560.5,...,0,1,1,0,1,0,0,0,0,0
1,307507,100005,NaN,0,0,0,0,99000.0,222768.0,17370.0,...,0,1,1,0,0,1,0,0,0,0
2,307508,100013,NaN,0,1,0,0,202500.0,663264.0,69777.0,...,0,1,1,0,0,1,1,0,1,0
3,307509,100028,NaN,1,0,0,2,315000.0,1575000.0,49018.5,...,0,1,1,0,0,1,1,0,1,0
4,307510,100038,NaN,0,1,1,1,180000.0,625500.0,32067.0,...,0,0,1,0,1,0,0,0,0,0


In [12]:
x_train = app_train.drop(columns=['TARGET', 'SK_ID_CURR'])
y_train = app_train['TARGET']
id_train = app_train[['SK_ID_CURR']]

for col in x_train.columns:
    if x_train[col].dtype=='O':
        x_train[col] = x_train[col].astype('category')

In [30]:
### パラメータ
"""
params = {
    'boosting_type': 'gbdt'
    ,'objective': 'binary'
    ,'metric': 'auc'
    ,'learning_rate': 0.05
    ,'num_leaves': 32
    ,'n_estimators': 100000
    ,'random_state':123
    ,'importance_type': 'gain'
}
"""

params = {
    'boosting_type': 'gbdt'
    ,'objective': 'binary'
    ,'metric': 'auc'
    ,'nthread': -1
    #,'device_type': 'gpu'
    ,'n_estimators': 5000
    ,'learning_rate': 0.01
    ,'max_depth': 11
    ,'num_leaves': 58
    ,'colsample_bytree': 0.613
    ,'subsample': 0.708
    ,'max_bin': 407
    ,'reg_alpha': 3.564
    ,'reg_lambda': 4.930
    ,'min_child_weight': 6
    ,'min_child_samples': 165
    #,'keep_training_booster': True
    ,'silent': -1
    ,'verbose': -1
    ,'importance_type': 'gain'
}


### 学習モデル構築
train_oof, imp, metrics = baseline.train_lgb(x_train
                                    ,y_train
                                    ,id_train
                                    ,params
                                    ,list_nfold=[0,1,2,3,4]
                                    ,n_splits=5)

-------------------- 0 --------------------
(246004, 1079) (61502, 1079)
[100]	training's auc: 0.7789	valid_1's auc: 0.755071
[200]	training's auc: 0.795763	valid_1's auc: 0.765712
[300]	training's auc: 0.80948	valid_1's auc: 0.773701
[400]	training's auc: 0.82057	valid_1's auc: 0.779694
[500]	training's auc: 0.829494	valid_1's auc: 0.783357
[600]	training's auc: 0.837285	valid_1's auc: 0.785932
[700]	training's auc: 0.844352	valid_1's auc: 0.787863
[800]	training's auc: 0.851005	valid_1's auc: 0.789421
[900]	training's auc: 0.857205	valid_1's auc: 0.790629
[1000]	training's auc: 0.863023	valid_1's auc: 0.791552
[1100]	training's auc: 0.868592	valid_1's auc: 0.792364
[1200]	training's auc: 0.873892	valid_1's auc: 0.792795
[1300]	training's auc: 0.879166	valid_1's auc: 0.793298
[1400]	training's auc: 0.884043	valid_1's auc: 0.793707
[1500]	training's auc: 0.888553	valid_1's auc: 0.794017
[1600]	training's auc: 0.892865	valid_1's auc: 0.794346
[1700]	training's auc: 0.896934	valid_1's au

In [31]:
imp.sort_values("imp", ascending=False)[:20]

,col,imp,imp_std
485,EXT_SOURCES_MEAN,347149.231815,24977.731682
135,APPS_EXT_SOURCE_MEAN,133847.087171,22103.661888
487,EXT_SOURCES_NANMEDIAN,55534.178185,3440.117053
771,PAYMENT_RATE,45306.358111,2272.946044
486,EXT_SOURCES_MIN,27156.334136,4715.753405
458,CREDIT_TO_GOODS_RATIO,23740.321646,1725.466913
183,BURO_BUREAU_CREDIT_DEBT_RATIO_MAX,21728.459592,2820.047532
146,APP_EXT_SOURCE_2_EXT_SOURCE_3,19443.357372,2736.525010
16,ACTIVE_BUREAU_CREDIT_DEBT_RATIO_MAX,19104.364546,3557.334104
150,APP_SCORE1_TO_BIRTH_RATIO,18343.083249,1305.715991


In [32]:
app_test.head()

,level_0,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CLOSED_CNT_CREDIT_PROLONG_SUM_high_risk,CLOSED_CNT_CREDIT_PROLONG_SUM_low_risk,PREV_PREV_GOODS_DIFF_MAX_high_risk,PREV_PREV_GOODS_DIFF_MAX_low_risk,POS_COMPLETED_BEFORE_MEAN_high_risk,POS_COMPLETED_BEFORE_MEAN_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk
0,307506,100001,NaN,1,0,0,0,135000.0,568800.0,20560.5,...,0,1,1,0,1,0,0,0,0,0
1,307507,100005,NaN,0,0,0,0,99000.0,222768.0,17370.0,...,0,1,1,0,0,1,0,0,0,0
2,307508,100013,NaN,0,1,0,0,202500.0,663264.0,69777.0,...,0,1,1,0,0,1,1,0,1,0
3,307509,100028,NaN,1,0,0,2,315000.0,1575000.0,49018.5,...,0,1,1,0,0,1,1,0,1,0
4,307510,100038,NaN,0,1,1,1,180000.0,625500.0,32067.0,...,0,0,1,0,1,0,0,0,0,0


In [33]:
# データセット作成
x_test = app_test.drop(columns=['SK_ID_CURR', 'TARGET'])
id_test = app_test[['SK_ID_CURR']]

# カテゴリ変数をcategory型へ変換
for col in x_test.columns:
    if x_test[col].dtype=="O":
        x_test[col] = x_test[col].astype('category')

In [34]:
test_pred = baseline.predict_lgb(x_test
                                ,id_test
                                ,list_nfold=[0,1,2,3,4])

-------------------- 0 --------------------
-------------------- 1 --------------------
-------------------- 2 --------------------
-------------------- 3 --------------------
-------------------- 4 --------------------
Done.


In [35]:
df_submit = test_pred.rename(columns={"pred":"TARGET"})
print(df_submit.shape)
display(df_submit.head())
df_submit.to_csv(
    path_or_buf="./home-credit-default-risk/submit/submission_featureEngineeringOtherSource2-1.csv" # 出力先
    ,index=None                                        # indexの出力有無
)

(48744, 2)


,SK_ID_CURR,TARGET
0,100001,0.031412
1,100005,0.112711
2,100013,0.043062
3,100028,0.032584
4,100038,0.172649


### 正規化＋対数変換

In [55]:
app_train_ord = app_train.copy()
app_test_ord = app_test.copy()
app_test_ord.head()

,level_0,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CLOSED_CNT_CREDIT_PROLONG_SUM_high_risk,CLOSED_CNT_CREDIT_PROLONG_SUM_low_risk,PREV_PREV_GOODS_DIFF_MAX_high_risk,PREV_PREV_GOODS_DIFF_MAX_low_risk,POS_COMPLETED_BEFORE_MEAN_high_risk,POS_COMPLETED_BEFORE_MEAN_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk
0,307506,100001,NaN,1,0,0,0,135000.0,568800.0,20560.5,...,0,1,1,0,1,0,0,0,0,0
1,307507,100005,NaN,0,0,0,0,99000.0,222768.0,17370.0,...,0,1,1,0,0,1,0,0,0,0
2,307508,100013,NaN,0,1,0,0,202500.0,663264.0,69777.0,...,0,1,1,0,0,1,1,0,1,0
3,307509,100028,NaN,1,0,0,2,315000.0,1575000.0,49018.5,...,0,1,1,0,0,1,1,0,1,0
4,307510,100038,NaN,0,1,1,1,180000.0,625500.0,32067.0,...,0,0,1,0,1,0,0,0,0,0


In [56]:
# infをnanへ置換
app_train_ord = app_train_ord.replace([np.inf, -np.inf], np.nan)
app_test_ord = app_test_ord.replace([np.inf, -np.inf], np.nan)
app_test_ord.head()

,level_0,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CLOSED_CNT_CREDIT_PROLONG_SUM_high_risk,CLOSED_CNT_CREDIT_PROLONG_SUM_low_risk,PREV_PREV_GOODS_DIFF_MAX_high_risk,PREV_PREV_GOODS_DIFF_MAX_low_risk,POS_COMPLETED_BEFORE_MEAN_high_risk,POS_COMPLETED_BEFORE_MEAN_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk
0,307506,100001,NaN,1,0,0,0,135000.0,568800.0,20560.5,...,0,1,1,0,1,0,0,0,0,0
1,307507,100005,NaN,0,0,0,0,99000.0,222768.0,17370.0,...,0,1,1,0,0,1,0,0,0,0
2,307508,100013,NaN,0,1,0,0,202500.0,663264.0,69777.0,...,0,1,1,0,0,1,1,0,1,0
3,307509,100028,NaN,1,0,0,2,315000.0,1575000.0,49018.5,...,0,1,1,0,0,1,1,0,1,0
4,307510,100038,NaN,0,1,1,1,180000.0,625500.0,32067.0,...,0,0,1,0,1,0,0,0,0,0


In [57]:
### 処理対象の列名を配列化
#encoding_columns = []
encoding_columns = list(app_test_ord.select_dtypes(include='number').columns)

#for col in df_test.columns:
#    print(col)
#    encoding_columns.append(col)
    
encoding_columns.remove('SK_ID_CURR')
encoding_columns.remove('TARGET')

In [58]:
### 欠損値補完
#app_train_ord = app_train_ord.fillna(app_train_ord.max() + (app_train_ord.max() * 0.4))
app_train_ord.head()

,level_0,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CLOSED_CNT_CREDIT_PROLONG_SUM_high_risk,CLOSED_CNT_CREDIT_PROLONG_SUM_low_risk,PREV_PREV_GOODS_DIFF_MAX_high_risk,PREV_PREV_GOODS_DIFF_MAX_low_risk,POS_COMPLETED_BEFORE_MEAN_high_risk,POS_COMPLETED_BEFORE_MEAN_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk
0,0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,...,0,1,1,0,1,0,0,0,0,0
1,1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,...,0,1,1,0,0,1,0,0,0,0
2,2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,...,0,1,1,0,0,1,0,0,0,0
3,3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,...,0,0,1,0,0,1,1,0,0,0
4,4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,...,0,1,1,0,0,1,0,0,0,0


In [59]:
### 欠損値補完
#app_test_ord = app_test_ord.fillna(app_test_ord.max() + (app_test_ord.max() * 0.4))
app_test_ord.head()

,level_0,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CLOSED_CNT_CREDIT_PROLONG_SUM_high_risk,CLOSED_CNT_CREDIT_PROLONG_SUM_low_risk,PREV_PREV_GOODS_DIFF_MAX_high_risk,PREV_PREV_GOODS_DIFF_MAX_low_risk,POS_COMPLETED_BEFORE_MEAN_high_risk,POS_COMPLETED_BEFORE_MEAN_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk
0,307506,100001,NaN,1,0,0,0,135000.0,568800.0,20560.5,...,0,1,1,0,1,0,0,0,0,0
1,307507,100005,NaN,0,0,0,0,99000.0,222768.0,17370.0,...,0,1,1,0,0,1,0,0,0,0
2,307508,100013,NaN,0,1,0,0,202500.0,663264.0,69777.0,...,0,1,1,0,0,1,1,0,1,0
3,307509,100028,NaN,1,0,0,2,315000.0,1575000.0,49018.5,...,0,1,1,0,0,1,1,0,1,0
4,307510,100038,NaN,0,1,1,1,180000.0,625500.0,32067.0,...,0,0,1,0,1,0,0,0,0,0


In [60]:
### Yao-Jhonson変換
#for column in encoding_columns:
#    modules.process_yj_convert(app_train_ord, app_test_ord, column, [column])#
#
#app_test_ord.head()

In [61]:
### indexを設定
app_train_ord = app_train_ord.set_index('SK_ID_CURR')
app_test_ord = app_test_ord.set_index('SK_ID_CURR')
app_test_ord.head()

,level_0,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,CLOSED_CNT_CREDIT_PROLONG_SUM_high_risk,CLOSED_CNT_CREDIT_PROLONG_SUM_low_risk,PREV_PREV_GOODS_DIFF_MAX_high_risk,PREV_PREV_GOODS_DIFF_MAX_low_risk,POS_COMPLETED_BEFORE_MEAN_high_risk,POS_COMPLETED_BEFORE_MEAN_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,307506,NaN,1,0,0,0,135000.0,568800.0,20560.5,450000.0,...,0,1,1,0,1,0,0,0,0,0
100005,307507,NaN,0,0,0,0,99000.0,222768.0,17370.0,180000.0,...,0,1,1,0,0,1,0,0,0,0
100013,307508,NaN,0,1,0,0,202500.0,663264.0,69777.0,630000.0,...,0,1,1,0,0,1,1,0,1,0
100028,307509,NaN,1,0,0,2,315000.0,1575000.0,49018.5,1575000.0,...,0,1,1,0,0,1,1,0,1,0
100038,307510,NaN,0,1,1,1,180000.0,625500.0,32067.0,625500.0,...,0,0,1,0,1,0,0,0,0,0


In [62]:
### 標準化
for column in encoding_columns:
    num_cols = []
    num_cols.append(column)
    result = modules.process_standardization(app_train_ord, app_test_ord, num_cols)
        
app_test_ord.head()

,level_0,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,CLOSED_CNT_CREDIT_PROLONG_SUM_high_risk,CLOSED_CNT_CREDIT_PROLONG_SUM_low_risk,PREV_PREV_GOODS_DIFF_MAX_high_risk,PREV_PREV_GOODS_DIFF_MAX_low_risk,POS_COMPLETED_BEFORE_MEAN_high_risk,POS_COMPLETED_BEFORE_MEAN_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,1.732056,NaN,0.720379,-0.717912,-0.664539,-0.577531,-0.307114,-0.075104,-0.448077,-0.232482,...,0.0,0.579736,0.245284,0.0,0.946577,-0.840113,-0.593453,-0.150513,-0.498057,0.0
100005,1.732068,NaN,-1.388158,-0.717912,-0.664539,-0.577531,-0.637959,-0.934826,-0.665962,-0.907160,...,0.0,0.579736,0.245284,0.0,-1.056438,1.190317,-0.593453,-0.150513,-0.498057,0.0
100013,1.732079,NaN,-1.388158,1.392928,-0.664539,-0.577531,0.313221,0.159594,2.913009,0.217303,...,0.0,0.579736,0.245284,0.0,-1.056438,1.190317,1.685055,-0.150513,2.007802,0.0
100028,1.732090,NaN,0.720379,-0.717912,-0.664539,2.192099,1.347112,2.424818,1.495373,2.578675,...,0.0,0.579736,0.245284,0.0,-1.056438,1.190317,1.685055,-0.150513,2.007802,0.0
100038,1.732102,NaN,-1.388158,1.392928,1.504802,0.807284,0.106443,0.065768,0.337723,0.206059,...,0.0,-1.724923,0.245284,0.0,0.946577,-0.840113,-0.593453,-0.150513,-0.498057,0.0


In [63]:
### indexを削除
app_train_ord = app_train_ord.reset_index()
app_test_ord = app_test_ord.reset_index()
app_test_ord.head()

,SK_ID_CURR,level_0,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CLOSED_CNT_CREDIT_PROLONG_SUM_high_risk,CLOSED_CNT_CREDIT_PROLONG_SUM_low_risk,PREV_PREV_GOODS_DIFF_MAX_high_risk,PREV_PREV_GOODS_DIFF_MAX_low_risk,POS_COMPLETED_BEFORE_MEAN_high_risk,POS_COMPLETED_BEFORE_MEAN_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk
0,100001,1.732056,NaN,0.720379,-0.717912,-0.664539,-0.577531,-0.307114,-0.075104,-0.448077,...,0.0,0.579736,0.245284,0.0,0.946577,-0.840113,-0.593453,-0.150513,-0.498057,0.0
1,100005,1.732068,NaN,-1.388158,-0.717912,-0.664539,-0.577531,-0.637959,-0.934826,-0.665962,...,0.0,0.579736,0.245284,0.0,-1.056438,1.190317,-0.593453,-0.150513,-0.498057,0.0
2,100013,1.732079,NaN,-1.388158,1.392928,-0.664539,-0.577531,0.313221,0.159594,2.913009,...,0.0,0.579736,0.245284,0.0,-1.056438,1.190317,1.685055,-0.150513,2.007802,0.0
3,100028,1.732090,NaN,0.720379,-0.717912,-0.664539,2.192099,1.347112,2.424818,1.495373,...,0.0,0.579736,0.245284,0.0,-1.056438,1.190317,1.685055,-0.150513,2.007802,0.0
4,100038,1.732102,NaN,-1.388158,1.392928,1.504802,0.807284,0.106443,0.065768,0.337723,...,0.0,-1.724923,0.245284,0.0,0.946577,-0.840113,-0.593453,-0.150513,-0.498057,0.0


In [64]:
x_train = app_train_ord.drop(columns=['TARGET', 'SK_ID_CURR'])
y_train = app_train_ord['TARGET']
id_train = app_train_ord[['SK_ID_CURR']]

for col in x_train.columns:
    if x_train[col].dtype=='O':
        x_train[col] = x_train[col].astype('category')

In [65]:
# データセット作成
x_test = app_test_ord.drop(columns=['SK_ID_CURR', 'TARGET'])
id_test = app_test_ord[['SK_ID_CURR']]

# カテゴリ変数をcategory型へ変換
for col in x_test.columns:
    if x_test[col].dtype=="O":
        x_test[col] = x_test[col].astype('category')

In [66]:
del app_train_ord
del app_test_ord
gc.collect()

10

In [67]:
### パラメータ
"""
params = {
    'boosting_type': 'gbdt'
    ,'objective': 'binary'
    ,'metric': 'auc'
    ,'learning_rate': 0.05
    ,'num_leaves': 32
    ,'n_estimators': 100000
    ,'random_state':123
    ,'importance_type': 'gain'
}
"""

params = {
    'boosting_type': 'gbdt'
    ,'objective': 'binary'
    ,'metric': 'auc'
    ,'nthread': -1
    #,'device_type': 'gpu'
    ,'n_estimators': 5000
    ,'learning_rate': 0.01
    ,'max_depth': 11
    ,'num_leaves': 58
    ,'colsample_bytree': 0.613
    ,'subsample': 0.708
    ,'max_bin': 407
    ,'reg_alpha': 3.564
    ,'reg_lambda': 4.930
    ,'min_child_weight': 6
    ,'min_child_samples': 165
    #,'keep_training_booster': True
    ,'silent': -1
    ,'verbose': -1
    ,'importance_type': 'gain'
}


### 学習モデル構築
train_oof, imp, metrics = baseline.train_lgb(x_train
                                    ,y_train
                                    ,id_train
                                    ,params
                                    ,list_nfold=[0,1,2,3,4]
                                    ,n_splits=5)

-------------------- 0 --------------------
(246004, 1079) (61502, 1079)
[LightGBM] [Warning] num_threads is set with n_jobs=-1, nthread=-1 will be ignored. Current value: num_threads=-1
[100]	training's auc: 0.77394	valid_1's auc: 0.75262
[200]	training's auc: 0.791768	valid_1's auc: 0.763929
[300]	training's auc: 0.805621	valid_1's auc: 0.772028
[400]	training's auc: 0.816544	valid_1's auc: 0.777801
[500]	training's auc: 0.825492	valid_1's auc: 0.781607
[600]	training's auc: 0.833507	valid_1's auc: 0.784416
[700]	training's auc: 0.840855	valid_1's auc: 0.786609
[800]	training's auc: 0.847639	valid_1's auc: 0.788329
[900]	training's auc: 0.853903	valid_1's auc: 0.789598
[1000]	training's auc: 0.859782	valid_1's auc: 0.790583
[1100]	training's auc: 0.865765	valid_1's auc: 0.791577
[1200]	training's auc: 0.871187	valid_1's auc: 0.792212
[1300]	training's auc: 0.876379	valid_1's auc: 0.792819
[1400]	training's auc: 0.881206	valid_1's auc: 0.793222
[1500]	training's auc: 0.885928	valid_1'

[2300]	training's auc: 0.916249	valid_1's auc: 0.799483
[2400]	training's auc: 0.919359	valid_1's auc: 0.799589
[2500]	training's auc: 0.922417	valid_1's auc: 0.799624
[2600]	training's auc: 0.9254	valid_1's auc: 0.799558
[auc] tr:0.921891, va0.7996
-------------------- result --------------------
[[0.         0.9394813  0.79577062]
 [1.         0.92655807 0.79840969]
 [2.         0.92696427 0.79299721]
 [3.         0.90389609 0.79822237]
 [4.         0.9218909  0.79963146]]
[cv] tr:0.9238+-0.0115, va:0.7970+-0.0024
[oof]0.7969


In [68]:
imp.sort_values("imp", ascending=False)[:20]

,col,imp,imp_std
485,EXT_SOURCES_MEAN,351780.973495,24294.161427
135,APPS_EXT_SOURCE_MEAN,135422.501741,21922.123074
487,EXT_SOURCES_NANMEDIAN,58156.340454,4588.955199
771,PAYMENT_RATE,50251.676748,2810.155339
486,EXT_SOURCES_MIN,29199.166662,5151.129699
458,CREDIT_TO_GOODS_RATIO,25474.664628,1646.073298
460,DAYS_BIRTH,22131.236113,643.920996
484,EXT_SOURCES_MAX,20468.109636,2313.183063
63,AMT_ANNUITY,18944.510918,1279.222083
183,BURO_BUREAU_CREDIT_DEBT_RATIO_MAX,18837.852779,2398.484510


In [69]:
test_pred = baseline.predict_lgb(x_test
                                ,id_test
                                ,list_nfold=[0,1,2,3,4])

-------------------- 0 --------------------
-------------------- 1 --------------------
-------------------- 2 --------------------
-------------------- 3 --------------------
-------------------- 4 --------------------
Done.


In [70]:
df_submit = test_pred.rename(columns={"pred":"TARGET"})
print(df_submit.shape)
display(df_submit.head())
df_submit.to_csv(
    path_or_buf="./home-credit-default-risk/submit/submission_featureEngineeringOtherSource2-2.csv" # 出力先
    ,index=None                                        # indexの出力有無
)

(48744, 2)


,SK_ID_CURR,TARGET
0,100001,0.040287
1,100005,0.106801
2,100013,0.044305
3,100028,0.032235
4,100038,0.255254


## パラメータチューニング

In [ ]:
sampler = optuna.samplers.TPESampler(seed=123)
study = optuna.create_study(sampler=sampler, direction='maximize')
study.optimize(objective, n_trials=50, n_jobs=5)

[I 2022-06-28 15:47:41,094] A new study created in memory with name: no-name-754aa063-95d2-45ff-ba98-193c41b06f20


In [ ]:
trial = study.best_trial
print("auc(best)={:.4f}".format(trial.value))
display(trial.params)

In [ ]:
params_best = trial.params
params_best.update(params_base)
display(params_best)

In [ ]:
train_oof, imp, metrics = baseline.train_lgb(x_train
                                    ,y_train
                                    ,id_train
                                    ,list_nfold=[0,1,2,3,4]
                                    ,n_splits=5
                                    ,params=params_best)

In [ ]:
# 推論用データセットの作成
x_test = df_test.drop(columns=['SK_ID_CURR'])
id_test = df_test[['SK_ID_CURR']]

# カテゴリ変数をcategory型へ変換
for col in x_test.columns:
    if x_test[col].dtype=="O":
        x_test[col] = x_test[col].astype('category')

# predict
test_pred = baseline.predict_lgb(x_test
                       ,id_test
                       ,list_nfold=[0,1,2,3,4])

# make submission-file
df_submit = test_pred.rename(columns={"pred":"TARGET"})
print(df_submit.shape)
display(df_submit.head())
df_submit.to_csv("./home-credit-default-risk/submit/submission_HyperParameterTuning.csv", index=None)

In [ ]:
del df_train
del df_test
gc.collect()